# PPE Compliance Detection Project

This notebook demonstrates the development of a deep learning model for detecting Personal Protective Equipment (PPE) compliance in construction environments. The goal is to classify images into multiple classes, including both compliant and non-compliant scenarios.

## Project Overview
- **Dataset**: Images of workers wearing or missing PPE (e.g., helmets, gloves, vests).
- **Objective**: Build a model to detect PPE compliance and identify missing equipment.
- **Approach**: Use transfer learning with a pre-trained MobileNetV2 model.

## Steps
1. Data Loading and Preprocessing
2. Model Definition and Compilation
3. Model Training with Callbacks
4. Evaluation and Visualization
5. Model Export for Deployment

## Data Preparation and Cleaning

In this section, we will prepare the dataset for training. This includes:
- Applying data augmentation techniques to improve model generalization.
- Normalizing the image data to ensure consistent input to the model.
- Use generators to load data from directories

In [ ]:
# Import Required Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
df = pd.read_csv('/content/ppe-compliance-detection/data/labels.csv')
df.head()

In [ ]:
# Load and Preprocess Data
# Define paths to the dataset directories
data_dir = '/content/ppe-compliance-detection/data'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Normalization for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Correct target size??????
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

## Exploratory Data Analysis (EDA)

In this section, we will:
- Visualize the distribution of classes in the dataset.
- Analyze the image dimensions and aspect ratios.
- Identify any potential issues, such as class imbalance or missing data.

In [ ]:
# Exploratory Data Analysis (EDA)
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize class distribution
class_labels = list(train_generator.class_indices.keys())
class_counts = train_generator.classes

plt.figure(figsize=(10, 6))
sns.countplot(x=class_counts, palette='viridis')
plt.title('Class Distribution')
plt.xlabel('Class Labels')
plt.ylabel('Frequency')
plt.xticks(ticks=range(len(class_labels)), labels=class_labels, rotation=45)
plt.show()

# Analyze image dimensions
image_shapes = [img.shape for img, _ in train_generator]
heights = [shape[0] for shape in image_shapes]
widths = [shape[1] for shape in image_shapes]

plt.figure(figsize=(10, 6))
plt.hist(heights, bins=20, alpha=0.7, label='Heights')
plt.hist(widths, bins=20, alpha=0.7, label='Widths')
plt.title('Image Dimensions Distribution')
plt.xlabel('Pixels')
plt.ylabel('Frequency')
plt.legend()
plt.show()

## Model Selection and Parameter Tuning

In this section, we will:
- Define the model architecture using transfer learning with MobileNetV2.
- Compile the model with appropriate loss functions and metrics.
- Tune hyperparameters such as learning rate, batch size, and number of epochs.

In [ ]:
# Model Selection and Parameter Tuning
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

## Model Training with Callbacks
In this section, we will:
- Includes early stopping and model checkpointing.
- Trains the model for up to 20 epochs.

In [ ]:
# Model Training with Callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', save_best_only=True)
]

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=callbacks
)

## Evaluation and Visualization:
In this section, we will:
- Plots training and validation accuracy/loss.
- Evaluates the model on the test set and prints the results.

In [ ]:
# Evaluation and Visualization
import matplotlib.pyplot as plt

# Plot training history
plt.figure(figsize=(12, 4))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

## Model Export for Deployment:
In this section, we will save the trained model as ppe_compliance_model.h5.

In [ ]:
# Model Export for Deployment

# Save the trained model
model.save('ppe_compliance_model.h5')
print("Model saved as 'ppe_compliance_model.h5'")